### Student Information
Name:丁寧

Student ID: 108134505

GitHub ID: niningding

Kaggle name: iamnining

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM19-Lab2-Master Repo](https://github.com/EvaArevalo/DM19-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/t/179d01d4dd984fc5ac45a894822479dd) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the score (ie. 20% of 30% )

    - **Top 41% - 100%**: Get (101-x)% of the score, where x is your ranking in the leaderboard (ie. (101-x)% of 30% )   
    Submit your last submission __BEFORE the deadline (Nov. 23rd 11:59 pm, Saturday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


You can submit your homework following these guidelines: [Git Intro & How to hand your homework](https://github.com/EvaArevalo/DM19-Lab1-Master/blob/master/Git%20Intro%20%26%20How%20to%20hand%20your%20homework.ipynb), but make sure to fork the [DM19-Lab2-Homework](https://github.com/EvaArevalo/DM19-Lab2-Homework) repository this time! Also please __DON´T UPLOAD HUGE DOCUMENTS__, please use Git ignore for that.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th 11:59 pm, Tuesday)__. 

## First part

The take home exercises is in another file called **DM_Lab2_Master**, thanks!

## Third part

Following is some part of my code for Kaggle Competition and then I will do some explanation.

### Load data

In [ ]:
tweets = pd.read_json("data/tweets_DM.json", lines = True)

# convert the data from .json into dataframe
tweets_df = pd.DataFrame()
tweets_df['tweet_id'] = [item['tweet']['tweet_id'] for item in tweets['_source']]
tweets_df['text'] = [item['tweet']['text'] for item in tweets['_source']]

emotion_df = pd.read_csv("data/emotion.csv")
ident_df = pd.read_csv("data/data_identification.csv")

### Split the tweet id into train & test set

In [ ]:
f_train_df = ident_df[ident_df['identification'] == 'train']
f_test_df = ident_df[ident_df['identification'] == 'test']

### Merge dataframes to get the format I want
training set：tweet_id, identification, text, emotion <br>
testing set：tweet_id, identification, text

In [ ]:
m1 = pd.merge(f_train_df, emotion_df, how = 'left', on = 'tweet_id')
m2 = pd.merge(m1, tweets_df, how = 'left', on = 'tweet_id')
train_df = m2.reindex(columns=['tweet_id','identification','text','emotion'])

In [ ]:
test_df = pd.merge(f_test_df, tweets_df, how = 'left', on = 'tweet_id')

### Feature engineering
using BoW and the number of features is 500

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=500) 

# apply analyzer to training data
BOW_500.fit(train_df['text'])

train_data_BOW_features_500 = BOW_500.transform(train_df['text'])

## check dimension
train_data_BOW_features_500.shape

### Train model and predict
using Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

## build Naive Bayes model
NB_model = MultinomialNB()

## training!
NB_model = NB_model.fit(X_train, y_train)

In [ ]:
## predict!
y_train_pred = NB_model.predict(X_train)
y_test_pred = NB_model.predict(X_test)

## Improvement

### Feature engineering
using tf-idf and remove stopwords

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_1000 = TfidfVectorizer(max_features=1000, stop_words='english')

tfidf_1000.fit(train_df['text'])

train_data_tfidf_features = tfidf_1000.transform(train_df['text'])
train_data_tfidf_features.shape
train_data_tfidf_features.toarray()

feature_names_1000 = tfidf_1000.get_feature_names()
feature_names_1000[100:110]

### Train model and predict
also using Naive Bayes <br><br>
Two basic assumptions of Naive Bayes: (1) Texts are independent to each other (2) Every feature is equally important <br>
So if I remove stopwords and do tf-idf, the accuracy would be higher.